# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.12.0 opencv-python mediapipe sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import winsound

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['BharathaNatyam', 'Kathak', 'Bhangra'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [9]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    #for sequence in range(1,no_sequences+1):
     for sequence in range(no_sequences):
        try: 
            #os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [13]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    winsound.Beep(2500,500)
                    cv2.waitKey(2000)
                    winsound.Beep(1000,1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [14]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

# 6. Preprocess Data and Create Labels and Features

In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
label_map = {label:num for num, label in enumerate(actions)}

In [17]:
label_map

{'BharathaNatyam': 0, 'Kathak': 1, 'Bhangra': 2}

In [18]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [19]:
# Ok to skip
np.array(sequences).shape

(90, 30, 1662)

In [20]:
# Ok to skip
np.array(labels).shape

(90,)

In [21]:
# Ok to skip
X = np.array(sequences)

In [22]:
# Ok to skip
X.shape

(90, 30, 1662)

In [23]:
# Ok to skip
y = to_categorical(labels).astype(int)

In [24]:
# Ok to skip
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [25]:
# Ok to skip
y_test.shape

(5, 3)

# 7. Build and Train LSTM Neural Network

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [11]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [12]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [44]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [45]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
3/3 [==============================] - 4s 58ms/step - loss: 1.7285 - categorical_accuracy: 0.2706
Epoch 2/1000
3/3 [==============================] - 0s 59ms/step - loss: 16.0983 - categorical_accuracy: 0.4000
Epoch 3/1000
3/3 [==============================] - 0s 59ms/step - loss: 35.8657 - categorical_accuracy: 0.2235
Epoch 4/1000
3/3 [==============================] - 0s 56ms/step - loss: 37.4467 - categorical_accuracy: 0.4118
Epoch 5/1000
3/3 [==============================] - 0s 57ms/step - loss: 123.9568 - categorical_accuracy: 0.3412
Epoch 6/1000
3/3 [==============================] - 0s 56ms/step - loss: 88.6325 - categorical_accuracy: 0.3412
Epoch 7/1000
3/3 [==============================] - 0s 57ms/step - loss: 65.1179 - categorical_accuracy: 0.4000
Epoch 8/1000
3/3 [==============================] - 0s 58ms/step - loss: 63.6941 - categorical_accuracy: 0.3882
Epoch 9/1000
3/3 [==============================] - 0s 57ms/step - loss: 677.3931 - categorical_accuracy

3/3 [==============================] - 0s 59ms/step - loss: 40.1982 - categorical_accuracy: 0.3529
Epoch 146/1000
3/3 [==============================] - 0s 59ms/step - loss: 55.0841 - categorical_accuracy: 0.3294
Epoch 147/1000
3/3 [==============================] - 0s 61ms/step - loss: 40.4878 - categorical_accuracy: 0.4118
Epoch 148/1000
3/3 [==============================] - 0s 61ms/step - loss: 37.3283 - categorical_accuracy: 0.4824
Epoch 149/1000
3/3 [==============================] - 0s 59ms/step - loss: 27.1610 - categorical_accuracy: 0.3529
Epoch 150/1000
3/3 [==============================] - 0s 59ms/step - loss: 52.5590 - categorical_accuracy: 0.3529
Epoch 151/1000
3/3 [==============================] - 0s 59ms/step - loss: 35.7430 - categorical_accuracy: 0.4000
Epoch 152/1000
3/3 [==============================] - 0s 61ms/step - loss: 41.9336 - categorical_accuracy: 0.3647
Epoch 153/1000
3/3 [==============================] - 0s 59ms/step - loss: 44.1277 - categorical_accura

3/3 [==============================] - 0s 61ms/step - loss: 67.5159 - categorical_accuracy: 0.3529
Epoch 218/1000
3/3 [==============================] - 0s 61ms/step - loss: 49.5696 - categorical_accuracy: 0.3647
Epoch 219/1000
3/3 [==============================] - 0s 61ms/step - loss: 52.4171 - categorical_accuracy: 0.4118
Epoch 220/1000
3/3 [==============================] - 0s 60ms/step - loss: 56.7986 - categorical_accuracy: 0.4118
Epoch 221/1000
3/3 [==============================] - 0s 61ms/step - loss: 41.3953 - categorical_accuracy: 0.3176
Epoch 222/1000
3/3 [==============================] - 0s 63ms/step - loss: 62.1537 - categorical_accuracy: 0.3765
Epoch 223/1000
3/3 [==============================] - 0s 61ms/step - loss: 49.3514 - categorical_accuracy: 0.4353
Epoch 224/1000
3/3 [==============================] - 0s 61ms/step - loss: 44.1722 - categorical_accuracy: 0.3882
Epoch 225/1000
3/3 [==============================] - 0s 62ms/step - loss: 40.8162 - categorical_accura

3/3 [==============================] - 0s 63ms/step - loss: 21.1124 - categorical_accuracy: 0.3882
Epoch 290/1000
3/3 [==============================] - 0s 62ms/step - loss: 42.5525 - categorical_accuracy: 0.3412
Epoch 291/1000
3/3 [==============================] - 0s 61ms/step - loss: 24.4509 - categorical_accuracy: 0.3529
Epoch 292/1000
3/3 [==============================] - 0s 60ms/step - loss: 27.9984 - categorical_accuracy: 0.3294
Epoch 293/1000
3/3 [==============================] - 0s 62ms/step - loss: 25.9880 - categorical_accuracy: 0.4353
Epoch 294/1000
3/3 [==============================] - 0s 62ms/step - loss: 32.8729 - categorical_accuracy: 0.3529
Epoch 295/1000
3/3 [==============================] - 0s 62ms/step - loss: 30.1500 - categorical_accuracy: 0.4118
Epoch 296/1000
3/3 [==============================] - 0s 63ms/step - loss: 20.3397 - categorical_accuracy: 0.4235
Epoch 297/1000
3/3 [==============================] - 0s 60ms/step - loss: 24.0158 - categorical_accura

3/3 [==============================] - 0s 62ms/step - loss: 6.6038 - categorical_accuracy: 0.4588
Epoch 362/1000
3/3 [==============================] - 0s 62ms/step - loss: 4.6189 - categorical_accuracy: 0.5059
Epoch 363/1000
3/3 [==============================] - 0s 63ms/step - loss: 4.5667 - categorical_accuracy: 0.4706
Epoch 364/1000
3/3 [==============================] - 0s 62ms/step - loss: 7.5725 - categorical_accuracy: 0.3882
Epoch 365/1000
3/3 [==============================] - 0s 60ms/step - loss: 5.9898 - categorical_accuracy: 0.4706
Epoch 366/1000
3/3 [==============================] - 0s 63ms/step - loss: 4.3150 - categorical_accuracy: 0.5176
Epoch 367/1000
3/3 [==============================] - 0s 63ms/step - loss: 4.1348 - categorical_accuracy: 0.3765
Epoch 368/1000
3/3 [==============================] - 0s 63ms/step - loss: 5.4631 - categorical_accuracy: 0.4941
Epoch 369/1000
3/3 [==============================] - 0s 61ms/step - loss: 10.1996 - categorical_accuracy: 0.43

3/3 [==============================] - 0s 64ms/step - loss: 4.3427 - categorical_accuracy: 0.4824
Epoch 506/1000
3/3 [==============================] - 0s 62ms/step - loss: 3.5031 - categorical_accuracy: 0.6118
Epoch 507/1000
3/3 [==============================] - 0s 62ms/step - loss: 4.0900 - categorical_accuracy: 0.5882
Epoch 508/1000
3/3 [==============================] - 0s 62ms/step - loss: 2.9155 - categorical_accuracy: 0.5412
Epoch 509/1000
3/3 [==============================] - 0s 62ms/step - loss: 3.3100 - categorical_accuracy: 0.5529
Epoch 510/1000
3/3 [==============================] - 0s 62ms/step - loss: 3.0550 - categorical_accuracy: 0.4353
Epoch 511/1000
3/3 [==============================] - 0s 61ms/step - loss: 3.5447 - categorical_accuracy: 0.5059
Epoch 512/1000
3/3 [==============================] - 0s 63ms/step - loss: 4.4694 - categorical_accuracy: 0.4941
Epoch 513/1000
3/3 [==============================] - 0s 63ms/step - loss: 4.0588 - categorical_accuracy: 0.505

3/3 [==============================] - 0s 65ms/step - loss: 2.8934 - categorical_accuracy: 0.5882
Epoch 650/1000
3/3 [==============================] - 0s 65ms/step - loss: 5.4132 - categorical_accuracy: 0.5059
Epoch 651/1000
3/3 [==============================] - 0s 66ms/step - loss: 3.0958 - categorical_accuracy: 0.6000
Epoch 652/1000
3/3 [==============================] - 0s 65ms/step - loss: 1.7686 - categorical_accuracy: 0.6118
Epoch 653/1000
3/3 [==============================] - 0s 74ms/step - loss: 1.5638 - categorical_accuracy: 0.6235
Epoch 654/1000
3/3 [==============================] - 0s 65ms/step - loss: 1.2578 - categorical_accuracy: 0.6706
Epoch 655/1000
3/3 [==============================] - 0s 66ms/step - loss: 1.8997 - categorical_accuracy: 0.6000
Epoch 656/1000
3/3 [==============================] - 0s 66ms/step - loss: 1.8022 - categorical_accuracy: 0.5765
Epoch 657/1000
3/3 [==============================] - 0s 66ms/step - loss: 1.7975 - categorical_accuracy: 0.600

3/3 [==============================] - 0s 62ms/step - loss: 3.7499 - categorical_accuracy: 0.5412
Epoch 794/1000
3/3 [==============================] - 0s 62ms/step - loss: 3.2482 - categorical_accuracy: 0.6235
Epoch 795/1000
3/3 [==============================] - 0s 61ms/step - loss: 3.8546 - categorical_accuracy: 0.6471
Epoch 796/1000
3/3 [==============================] - 0s 62ms/step - loss: 3.4690 - categorical_accuracy: 0.6118
Epoch 797/1000
3/3 [==============================] - 0s 64ms/step - loss: 3.8675 - categorical_accuracy: 0.5765
Epoch 798/1000
3/3 [==============================] - 0s 62ms/step - loss: 3.9068 - categorical_accuracy: 0.5882
Epoch 799/1000
3/3 [==============================] - 0s 65ms/step - loss: 2.2733 - categorical_accuracy: 0.5765
Epoch 800/1000
3/3 [==============================] - 0s 63ms/step - loss: 1.8528 - categorical_accuracy: 0.6353
Epoch 801/1000
3/3 [==============================] - 0s 62ms/step - loss: 1.0898 - categorical_accuracy: 0.647

3/3 [==============================] - 0s 65ms/step - loss: 1.1294 - categorical_accuracy: 0.6941
Epoch 938/1000
3/3 [==============================] - 0s 74ms/step - loss: 0.9319 - categorical_accuracy: 0.6706
Epoch 939/1000
3/3 [==============================] - 0s 73ms/step - loss: 1.0552 - categorical_accuracy: 0.6588
Epoch 940/1000
3/3 [==============================] - 0s 66ms/step - loss: 1.1838 - categorical_accuracy: 0.6353
Epoch 941/1000
3/3 [==============================] - 0s 66ms/step - loss: 1.7085 - categorical_accuracy: 0.6118
Epoch 942/1000
3/3 [==============================] - 0s 67ms/step - loss: 1.5869 - categorical_accuracy: 0.5529
Epoch 943/1000
3/3 [==============================] - 0s 67ms/step - loss: 1.4723 - categorical_accuracy: 0.5882
Epoch 944/1000
3/3 [==============================] - 0s 66ms/step - loss: 1.3618 - categorical_accuracy: 0.6118
Epoch 945/1000
3/3 [==============================] - 0s 68ms/step - loss: 1.5108 - categorical_accuracy: 0.623

In [46]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

# 8. Make Predictions

In [47]:
res = model.predict(X_test)

1/1 [==============================] - 0s 329ms/step


In [48]:
actions[np.argmax(res[4])]

'Kathak'

In [49]:
actions[np.argmax(y_test[4])]

'Kathak'

# 9. Save Weights

In [50]:
model.save('action.h5')

In [13]:
del model

In [14]:
# Once training is done, u do not have to train it again (no need to call the fit() finction). Just need to setup the model
# and compile it. then load the saved weights.
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [16]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [17]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [18]:
yhat = model.predict(X_test)

NameError: name 'X_test' is not defined

In [19]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

NameError: name 'y_test' is not defined

In [20]:
multilabel_confusion_matrix(ytrue, yhat)

NameError: name 'ytrue' is not defined

In [21]:
accuracy_score(ytrue, yhat)

NameError: name 'ytrue' is not defined

# 11. Test in Real Time

In [22]:
from scipy import stats

In [23]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [24]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'res' is not defined

<Figure size 1800x1800 with 0 Axes>

In [28]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        #cv2.putText(image, ' '.join(sentence), (3,30), 
        #               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 14ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Kathak
<class 'medi

1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<cla

1/1 [==============================] - 0s 18ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Kathak
<class 'medi

1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
Bhangra
<class

...

1/1 [==============================] - 0s 18ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 18ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<cla

1/1 [==============================] - 0s 16ms/step
BharathaNatyam
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
BharathaNatyam
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
BharathaNatyam
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
BharathaNatyam
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
BharathaNatyam
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
BharathaNatyam
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [=================

1/1 [==============================] - 0s 24ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 22ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Kathak
<clas

1/1 [==============================] - 0s 16ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Kathak
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Kathak
<class 'm

1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
Bhangra
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
Bhangra
<cla